In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import json

Set the first and last month you are interested in to download the data:

In [ ]:
first_date = 200803
last_date = 202109

Get your credentials to access Google Big Query. We provide an example document below:

In [ ]:
path = 'data/credentials_bigquery_example.json'

f = open(path)

to_get_proj = json.load(f)

In [ ]:
project_id = to_get_proj.get('project_id')

In [ ]:
credentials = service_account.Credentials.from_service_account_file(path)

In [ ]:
query_string = f"""
SELECT ActionGeo_CountryCode, MonthYear, EventBaseCode,
count(EventBaseCode) as eventcount
FROM `gdelt-bq.full.events`
WHERE (MonthYear>={first_date} and MonthYear<={last_date}) and (ActionGeo_CountryCode <> 'null')
group by ActionGeo_CountryCode, MonthYear, EventBaseCode
Order by ActionGeo_CountryCode, EventBaseCode, MonthYear"""

In [ ]:
df = pandas_gbq.read_gbq(query_string, project_id=project_id, credentials = credentials)

Get the unique country codes:

In [ ]:
countries = df['ActionGeo_CountryCode'].unique()

Get the length of the unique country codes:

In [ ]:
len(countries)

Get the countries of the old gdelt data so that you append to the older data the new data:

In [ ]:
countries_old = []
for j in os.listdir('data/gdelt_data/'):
    if j != '.DS_Store':
        countries_old.append(j.split(".")[0])

In [ ]:
len(countries_old)

Append new data to older data and save:

In [ ]:
for c in countries_old:
    single_country = df.loc[df.ActionGeo_CountryCode == c]
    single_country.to_csv('data/gdelt_data/%s.csv' %c, mode='a', index=True, header=False)        